<a href="https://colab.research.google.com/github/DigoLazarini/Colab/blob/main/Projeto_Final_Alura_%2B_Google_Agentes_IA_com_Gemini_Criador_de_Loadouts_COD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [194]:
%pip -q install google-genai

In [195]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [196]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [197]:
from IPython.display import HTML, Markdown, display
import textwrap
import requests
import warnings
warnings.filterwarnings("ignore")
from datetime import date

In [198]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [199]:
!pip install -q youtube_search
import youtube_search

In [200]:
from google.adk.tools.base_tool import BaseTool
from google.adk.tools import google_search as google_search_tool
from youtube_search import YoutubeSearch # Importe a classe YoutubeSearch
from youtube_search import YoutubeSearch as YoutubeSearchTool # Mantenha o alias para usar como Tool


In [201]:
from google.adk.agents import Agent, BaseAgent # Importar Agent e BaseAgent
from google.adk.runners import Runner
from google.adk.tools import google_search as google_search_tool
from google.adk.tools.base_tool import BaseTool
from google.adk.sessions import InMemorySessionService
from google.genai import types
from youtube_search import YoutubeSearch, YoutubeSearch as YoutubeSearchTool

In [202]:
import google.adk.tools
print(dir(google.adk.tools))

['APIHubToolset', 'AuthToolArguments', 'BaseTool', 'ExampleTool', 'FunctionTool', 'LongRunningFunctionTool', 'ToolContext', 'VertexAiSearchTool', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_automatic_function_calling_util', 'apihub_tool', 'base_tool', 'built_in_code_execution', 'built_in_code_execution_tool', 'example_tool', 'exit_loop', 'exit_loop_tool', 'function_parameter_parse_util', 'function_tool', 'get_user_choice', 'get_user_choice_tool', 'google_api_tool', 'google_search', 'google_search_tool', 'load_artifacts', 'load_artifacts_tool', 'load_memory', 'load_memory_tool', 'long_running_tool', 'openapi_tool', 'preload_memory', 'preload_memory_tool', 'tool_context', 'transfer_to_agent', 'transfer_to_agent_tool', 'vertex_ai_search_tool']


In [203]:
class GoogleSearchTool(BaseTool):
    def __init__(self, **kwargs):
        super().__init__(name="google_search", description="Use para pesquisar informações no Google.", **kwargs)

    def _run(self, query: str) -> str:
        results = google_search_tool.search(query)
        return results

class YoutubeSearchTool(BaseTool):
    def __init__(self, **kwargs):
        super().__init__(name="youtube_search", description="Use para pesquisar vídeos no YouTube.", **kwargs)

    def _run(self, query: str) -> str:
        results = YoutubeSearch(search_terms=query, max_results=5).to_dict() # Use YoutubeSearch e search_terms
        formatted_results = "\n".join([f"Título: {r['title']}, Link: {r['url']}" for r in results])
        return formatted_results

In [204]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
            final_response += "\n"
    return final_response

def to_markdown(text):
    text = text.replace('.', '* ')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [205]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [206]:
##################################################
# --- Agente 1: Buscador de Builds (CoD) --- #
###################################################

def agente_buscador_cod(nome_da_arma: str, data_de_hoje: str, nome_jogo: str = "Call of Duty Black Ops 6 e Warzone") -> str:
    """
    Agente responsável por buscar builds de armas para Call of Duty
    no Google Search e YouTube.
    """
    print(f"--- Agente Buscador CoD: Buscando builds para {nome_da_arma}' em {nome_jogo}') ---")

    # Instanciando as ferramentas!
    google_search_instance = GoogleSearchTool()
    youtube_search_instance = YoutubeSearchTool()

    buscador_cod = Agent(
        name="agente_buscador_cod",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um assistente de pesquisa especializado em jogos Call of Duty.
        Sua tarefa é usar as ferramentas de busca do Google (google_search) e YouTube (youtube_search)
        para encontrar as builds de armas e equipamentos (attachments, perks) mais recentes e populares
        para a arma {nome_da_arma} no jogo {nome_jogo}.
        Concentre-se em encontrar vídeos e guias que detalham os acessórios e vantagens recomendados.
        Busque por termos como "melhor build {nome_da_arma} {nome_jogo}", "best {nome_da_arma} class setup {nome_jogo}",
        "meta {nome_da_arma} loadout {nome_jogo}".
        Forneça um resumo dos resultados encontrados, incluindo títulos de vídeos/guias e links diretos se possível.
        Priorize informações recentes (últimos 3 meses).
        """,
        description="Agente que busca builds de armas para Call of Duty no Google e YouTube.",
        tools=[google_search_instance, youtube_search_instance] # Passando as INSTÂNCIAS das ferramentas
    )

    entrada_do_agente_buscador = f"Arma: {nome_da_arma}\nJogo: {nome_jogo}\n"
    try:
        # Supondo que 'call_agent' seja uma função definida em outro lugar no seu código
        resultados_busca = call_agent(buscador_cod, entrada_do_agente_buscador)
        print("--- Agente Buscador CoD: Busca concluída ---")
        return resultados_busca
    except Exception as e:
        print(f"Erro no Agente Buscador CoD: {e}")
        return f"Erro ao buscar builds para {nome_da_arma}."

In [207]:
#########################################
# --- Agente 2: Planejador de Builds --- #
#########################################

def agente_planejador(lancamentos_buscados: str, nome_da_arma: str, nome_jogo: str) -> str:
    """
    Agente responsável por planejar builds com base nos resultados da busca.
    """
    print(f"--- Agente Planejador: Planejando builds para {nome_da_arma}' em {nome_jogo} ---")

    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um planejador de Builds, especialista em Criar Sets de Builds e Equips.
        Com base nos resultados de pesquisa fornecidos:

        {lancamentos_buscados}

        sua tarefa é identificar e extrair as melhores builds de armas para a arma {nome_da_arma} no jogo {nome_jogo}.
        Para cada build encontrada, liste os acessórios e as Vantagens (Perks) mencionados.

        Organize as informações de forma clara.
        """,
        description="Agente que planeja Builds",
        tools=[google_search_tool, YoutubeSearchTool()] # Corrigido para usar a instância da classe
    )

    entrada_do_agente_planejador = f"Resultados da busca: \n{lancamentos_buscados}"

    try:
        plano_de_build = call_agent(planejador, entrada_do_agente_planejador)
        print("--- Agente Planejador: Planejamento concluido---")
        return plano_de_build
    except Exception as e:
        print(f"Erro no Agente Planejador: {e}")
        return "Erro ao planejar builds."


In [208]:
########################################
# --- Agente 3: Comparador das Builds --- #
##########################################

def agente_comparador(builds_planejadas: str, modo_jogo: str) -> str:
    """
    Agente responsável por comparar as builds planejadas e selecionar as melhores
    com base no modo de jogo.
    """
    print(f"--- Agente Comparador: Comparando builds para o modo {modo_jogo} ---")

    comparador = Agent(
        name="agente_comparador_build",
        model="gemini-2.0-flash",
        instruction=f"""
        Voce é um Comparador de Builds, especialista em pegar os melhores equipamentos, vantagens e armas, e criar o melhor Set de Loadout.
        Com base nas builds fornecidas:

        {builds_planejadas}

        Identificar e extrair as 3 (três) melhores e mais distintas builds de armas adequadas para o modo de jogo {modo_jogo}.
        Para cada build, liste os acessórios (Ex: Boca, Cano, Laser, Mira, Coronha, Acoplamento, Pente/Tambor, Munição, Cabo Traseiro) e Vantagens (Perks).
        Se diferentes fontes sugerirem builds muito similares, tente consolidar ou priorizar a mais completa/recente.

        Se não encontrar informações suficientes para 3 builds distintas ou detalhadas, liste o que encontrar,
        indicando se a informação é limitada.

        Foque em extrair os nomes dos acessórios e vantagens.
        """,
        description="Agente comparador de builds para determinar a melhor combinaçao de loadout por modo de jogo."
    )

    entrada_do_agente_comparador = f"Builds planejadas: \n{builds_planejadas}\nModo de jogo: {modo_jogo}"

    try:
        rascunho = call_agent(comparador, entrada_do_agente_comparador)
        print("--- Agente Comparador: Comparação concluida ---")
        return rascunho
    except Exception as e:
        print(f"Erro no Agente Comparador: {e}")
        return "Erro ao comparar builds."

In [209]:
#################################
# --- Agente 4: Indicador de Loadout --- #
##########################################
#
def agente_indicador(builds_comparadas: str, modo_jogo: str) -> str:
    """
    Agente responsável por formatar as melhores builds como um loadout final.
    """
    print(f"--- Agente Indicador: Gerando loadout para o modo {modo_jogo} ---")

    indicador = Agent(
        name="agente_indicador",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um Editor e Revisor de Loadout meticuloso, especializado em Builds para os modos de jogo, Multplayer, Zombie e Warzone.
        Com base nas builds comparadas fornecidas:

        {builds_comparadas}

        Revise e faça a melhor combinação de loadout para o modo de jogo do player: {modo_jogo}.
        Apresente a informação de forma organizada, como uma lista, usando Markdown para formatação.

        Siga este formato:

        Build [Número]: [Nome da Build ou Fonte, se disponivel]**
        Boca: [Nome do Acessório]
        Cano: [Nome do Acessório]
        Laser: [Nome do Acessório]
        Mira: [Nome do Acessório]
        Coronha: [Nome do Acessórioj
        Acoplamento: [Nome do Acessório]
        Pente/Tambor: [Nome do Acessório] # Adicionado Pente/Tambor
        Munição: [Nome do Acessório]
        Cabo Traseiro: [Nome do Acessório]

        Vantagens Recomendadas: [Perk1], [Perk2], [Perk3],

        Responda APENAS com a lista das top 3 builds formatadas. Não adicione introduções ou conclusões extras.

        Se não houver informações suficientes para 3 builds completas, formate o que for possivel com as informações disponíveis.
        """,
        description="Agente indicador de loadout para as Builds."
    )

    entrada_do_indicador_loadout = f"Builds comparadas: \n{builds_comparadas}\nModo de jogo: {modo_jogo}"

    try:
        texto_revisado = call_agent(indicador, entrada_do_indicador_loadout)
        print("--- Agente Indicador: Loadout gerado ---")
        return texto_revisado
    except Exception as e:
        print(f"Erro no Agente Indicador: {e}")
        return "Erro ao gerar loadout."

In [210]:
print("Iniciando o Sistema de Criação de Builds e Loadouts para Call Of Duty: Black ops 6 e Warzone, com 4 Agentes")

# Obter o Tópico do Usuário
topico = input("❓ Por favor, digite a arma e o modo de jogo (Ex: M4 para Warzone) que deseja jogar: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar!")
else:
    print(f"Maravilha! Vamos então criar o seu Loadout para: {topico}")

    # Processamento com a cadeia de Agentes
    # 1. Agente Buscador CoD
    # Assuming the user input 'topico' can be split into weapon and mode.
    # A more robust parsing might be needed depending on user input variations.
    try:
        # Attempt to split the input into weapon and mode, or use the whole topic as weapon
        parts = topico.split(" para ")
        if len(parts) == 2:
            nome_da_arma = parts[0].strip()
            nome_jogo = parts[1].strip()
        else:
            nome_da_arma = topico.strip()
            nome_jogo = "Call of Duty Black Ops 6 e Warzone"


        lancamentos_buscados = agente_buscador_cod(nome_da_arma=nome_da_arma, data_de_hoje=data_de_hoje, nome_jogo=nome_jogo)
        print("\n--- 📝 Resultado do Agente 1 (agente_buscador_cod) ---\n")
        display(to_markdown(lancamentos_buscados))
        print("--------------------------------------------------------------")

        # 2. Agente Planejador
        # Pass the results from the search agent to the planner
        builds_planejadas = agente_planejador(lancamentos_buscados=lancamentos_buscados, nome_da_arma=nome_da_arma, nome_jogo=nome_jogo)
        print("\n--- 📝 Resultado do Agente 2 (agente_planejador) ---\n")
        display(to_markdown(builds_planejadas))
        print("--------------------------------------------------------------")

        # 3. Agente Comparador
        # Pass the planned builds and the game mode to the comparator
        # Assuming the user input 'topico' also contains the mode of game
        # A more robust way to get the game mode might be needed.
        modo_jogo = parts[1].strip() if len(parts) == 2 else "geral" # Default mode if not specified

        builds_comparadas = agente_comparador(builds_planejadas=builds_planejadas, modo_jogo=modo_jogo)
        print("\n--- 📝 Resultado do Agente 3 (agente_comparador) ---\n")
        display(to_markdown(builds_comparadas))
        print("--------------------------------------------------------------")


        # 4. Agente Indicador
        # Pass the compared builds and the game mode to the indicator
        post_final = agente_indicador(builds_comparadas=builds_comparadas, modo_jogo=modo_jogo)
        print("\n--- 📝 Resultado do Agente 4 (agente_indicador) ---\n")
        display(to_markdown(post_final))
        print("--------------------------------------------------------------")

    except Exception as e:
        print(f"Ocorreu um erro durante a execução dos agentes: {e}")

Iniciando o Sistema de Criação de Builds e Loadouts para Call Of Duty: Black ops 6 e Warzone, com 4 Agentes
❓ Por favor, digite a arma e o modo de jogo (Ex: M4 para Warzone) que deseja jogar: ppsh multplayer
Maravilha! Vamos então criar o seu Loadout para: ppsh multplayer
--- Agente Buscador CoD: Buscando builds para ppsh multplayer' em Call of Duty Black Ops 6 e Warzone') ---
--- Agente Buscador CoD: Busca concluída ---

--- 📝 Resultado do Agente 1 (agente_buscador_cod) ---



> Ok, vou buscar as melhores builds de PPSh para o multplayer de Call of Duty Black Ops 6 e Warzone, focando em vídeos e guias recentes que detalham os acessórios e vantagens recomendadas.
> 


--------------------------------------------------------------
--- Agente Planejador: Planejando builds para ppsh multplayer' em Call of Duty Black Ops 6 e Warzone ---
--- Agente Planejador: Planejamento concluido---

--- 📝 Resultado do Agente 2 (agente_planejador) ---



> Ok, estou pronto para procurar as melhores builds de PPSh para Call of Duty Black Ops 6 e Warzone. Assim que tiver os resultados da pesquisa com os acessórios e vantagens recomendadas, organizarei as informações de forma clara para você.
> 


--------------------------------------------------------------
--- Agente Comparador: Comparando builds para o modo geral ---
--- Agente Comparador: Comparação concluida ---

--- 📝 Resultado do Agente 3 (agente_comparador) ---



> Ok, compreendido! Assim que tiver os resultados da pesquisa com os acessórios e vantagens recomendadas para a PPSh, organizarei as informações em um formato de fácil leitura, focando em identificar e extrair as 3 melhores e mais distintas builds para o modo de jogo geral. Priorizarei os nomes dos acessórios e vantagens para cada build.
> 


--------------------------------------------------------------
--- Agente Indicador: Gerando loadout para o modo geral ---
--- Agente Indicador: Loadout gerado ---

--- 📝 Resultado do Agente 4 (agente_indicador) ---



> **Build 1: Equilibrada para Multijogador**
> Boca: Silenciador GRU
> Cano: Cano de 15.7" Task Force
> Laser: Laser VK Poderoso de 5mW
> Coronha: Coronha Esqueleto Wire
> Acoplamento: Punho Frontal Spetsnaz
> Pente/Tambor: Tambor de 55 projéteis
> Munição: [Não especificado nas fontes]
> Cabo Traseiro: [Não especificado nas fontes]
> 
> Vantagens Recomendadas: [Não especificado nas fontes]
> 
> **Build 2: Focada em mobilidade e agilidade**
> Boca: [Não especificado nas fontes]
> Cano: [Não especificado nas fontes]
> Laser: Laser Tiger Team Spotlight
> Mira: [Não especificado nas fontes]
> Coronha: [Não especificado nas fontes]
> Acoplamento: Empunhadura de Combate Bruiser
> Pente/Tambor: [Não especificado nas fontes]
> Munição: [Não especificado nas fontes]
> Cabo Traseiro: Empunhadura Serpent
> 
> Vantagens Recomendadas: [Não especificado nas fontes]
> 
> **Build 3: Precisão a distância**
> Boca: Estabilizador de Boca
> Cano: [Não especificado nas fontes]
> Laser: [Não especificado nas fontes]
> Mira: Reflexo Microflex LED
> Coronha: [Não especificado nas fontes]
> Acoplamento: [Não especificado nas fontes]
> Pente/Tambor: [Não especificado nas fontes]
> Munição: [Não especificado nas fontes]
> Cabo Traseiro: [Não especificado nas fontes]
> 
> Vantagens Recomendadas: [Não especificado nas fontes]
> 


--------------------------------------------------------------


In [ ]:
# prompt: crie uma interface WEB simples com a tematica do COD

# Para criar uma interface web simples no Colab, usamos bibliotecas como `ipywidgets` ou `gradio`.
# Gradio é mais fácil para criar interfaces web rápidas.

!pip install -q gradio

import gradio as gr

def processar_build(arma_input, modo_input):
    """
    Função que encapsula a lógica de execução dos agentes.
    """
    print(f"Processando requisição: Arma={arma_input}, Modo={modo_input}")
    try:
        # Execução da cadeia de Agentes
        data_de_hoje = date.today().strftime("%d/%m/%Y") # Obter data atual para passar aos agentes

        lancamentos_buscados = agente_buscador_cod(nome_da_arma=arma_input, data_de_hoje=data_de_hoje, nome_jogo=f"Call of Duty Black Ops 6 e Warzone - Modo {modo_input}")
        print("\n--- Agente 1 (agente_buscador_cod) Concluído ---\n")

        builds_planejadas = agente_planejador(lancamentos_buscados=lancamentos_buscados, nome_da_arma=arma_input, nome_jogo=f"Call of Duty Black Ops 6 e Warzone - Modo {modo_input}")
        print("\n--- Agente 2 (agente_planejador) Concluído ---\n")

        builds_comparadas = agente_comparador(builds_planejadas=builds_planejadas, modo_jogo=modo_input)
        print("\n--- Agente 3 (agente_comparador) Concluído ---\n")

        post_final = agente_indicador(builds_comparadas=builds_comparadas, modo_jogo=modo_input)
        print("\n--- Agente 4 (agente_indicador) Concluído ---\n")

        return post_final # Retornar o resultado final do indicador
    except Exception as e:
        print(f"Erro durante o processamento da build: {e}")
        return f"Ocorreu um erro ao processar sua solicitação: {e}"

# Configuração da Interface Web com Gradio
with gr.Blocks(theme=gr.themes.Soft(primary_hue="red")) as demo:
    gr.Markdown(
        """
        <h1 style='text-align: center; color: #e30000;'>🛠️ Criador de Loadouts para Call of Duty 🎮</h1>
        <p style='text-align: center;'>Encontre as melhores builds e equipamentos para suas armas favoritas no Warzone e Black Ops 6!</p>
        """
    )

    with gr.Row():
        arma_input = gr.Textbox(label="Nome da Arma (Ex: M4, KV Broadside)")
        modo_input = gr.Dropdown(
            choices=["Warzone Battle Royale", "Warzone Ressurgência", "Multiplayer", "Zombies"],
            label="Modo de Jogo"
        )

    submit_button = gr.Button("Criar Loadout!")

    gr.Markdown(
        """
        <h2 style='text-align: center;'>Resultado do Loadout</h2>
        """
    )
    output_text = gr.Markdown()

    submit_button.click(
        fn=processar_build,
        inputs=[arma_input, modo_input],
        outputs=output_text
    )

demo.launch(debug=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://41716a48244ff8a3bb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Processando requisição: Arma=ppsh, Modo=Warzone Battle Royale
--- Agente Buscador CoD: Buscando builds para ppsh' em Call of Duty Black Ops 6 e Warzone - Modo Warzone Battle Royale') ---
--- Agente Buscador CoD: Busca concluída ---

--- Agente 1 (agente_buscador_cod) Concluído ---

--- Agente Planejador: Planejando builds para ppsh' em Call of Duty Black Ops 6 e Warzone - Modo Warzone Battle Royale ---
--- Agente Planejador: Planejamento concluido---

--- Agente 2 (agente_planejador) Concluído ---

--- Agente Comparador: Comparando builds para o modo Warzone Battle Royale ---
--- Agente Comparador: Comparação concluida ---

--- Agente 3 (agente_comparador) Concluído ---

--- Agente Indicador: Gerando loadout para o modo Warzone Battle Royale ---
--- Agente Indicador: Loadout gerado ---

--- Agente 4 (agente_indicador) Concluído ---

Processando requisição: Arma=xm4, Modo=Warzone Battle Royale
--- Agente Buscador CoD: Buscando builds para xm4' em Call of Duty Black Ops 6 e Warzone - Mod